## Библиотеки

In [6]:
import pandas as pd
import datetime as dt

## Данные

In [7]:
# Данные о клиентах
customers = pd.read_csv('C:/Users/Arthur/Desktop/customers.csv')

In [8]:
# Данные о заказах
orders = pd.read_csv('C:/Users/Arthur/Desktop/orders.csv',
                     parse_dates=['order_purchase_timestamp'])

In [9]:
# Данные о составах заказов
order_items = pd.read_csv('C:/Users/Arthur/Desktop/order_items.csv')

## Задания

### Задание 1

При помощи имеющихся данных необходимо сформулировать,  
что является покупкой, а также ответить на вопрос:  
Сколько пользователей, совершивших только одну покупку?

Объединим датафрейм с данными о клиентах и о заказах, т.к. информация об  
уникальных пользователях хранится в первой таблице, а покупки мы найдём  
во второй.

In [10]:
customers_orders = customers.merge(orders, on='customer_id', how='left')

In [11]:
customers_orders.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00


Рассмотрим статусы заказов.

In [12]:
customers_orders.order_status.value_counts()

order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64

Что такое "покупка"? Покупка — оплаченный заказ, который может быть доставлен  
и который не отменен. Следовательно нам не подходят следующие статусы заказов:  
"created" — заказ только создан и не оплачен;  
"invoiced" — выставлен счёт по заказу, ожидается оплата;  
"unavailable" — товар отсутствует/не может быть доставлен;  
"canceled" — заказ отменён.

In [13]:
unacceptable_status = ['created', 'invoiced', 'unavailable', 'canceled']
purchases = customers_orders.query('order_status not in @unacceptable_status')

Проверим, остались ли в отфильтрованном датафрейме заказы с неподтверждённой  
оплатой, если это так — удалим их.

In [14]:
purchases.order_approved_at.isna().sum()

14

In [15]:
drop_unapproved = purchases[['customer_id', 'order_approved_at']].dropna().customer_id
purchases = purchases.query('customer_id in @drop_unapproved')

In [16]:
purchases.order_approved_at.isna().sum()

0

Найдём число пользователей с одной покупкой.

In [17]:
customers_with_one_purchase = purchases.groupby('customer_unique_id', as_index=False) \
                                       .agg({'customer_id' : 'count'}) \
                                       .rename(columns={'customer_id' : 'purchases'}) \
                                       .query('purchases == 1')

In [18]:
customers_with_one_purchase['purchases'].count()

91804

### Задание 2

Сколько заказов в месяц в среднем не доставляется по разным причинам  
(вывести детализацию по причинам)?

Создадим с недоставленными заказами (т.е. не со статусом "delivered").

In [19]:
undelivered_orders = orders.query('order_status != "delivered"')

In [20]:
undelivered_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaN,NaN,2017-05-09 00:00:00
44,ee64d42b8cf066f35eac1cf57de1aa85,caded193e8e47b8362864762a83db3c5,shipped,2018-06-04 16:44:48,2018-06-05 04:31:18,2018-06-05 14:32:00,NaN,2018-06-28 00:00:00
103,0760a852e4e9d89eb77bf631eaaf1c84,d2a79636084590b7465af8ab374a8cf5,invoiced,2018-08-03 17:44:42,2018-08-07 06:15:14,NaN,NaN,2018-08-21 00:00:00
128,15bed8e2fec7fdbadb186b57c46c92f2,f3f0e613e0bdb9c7cee75504f0f90679,processing,2017-09-03 14:22:03,2017-09-03 14:30:09,NaN,NaN,2017-10-03 00:00:00
154,6942b8da583c2f9957e990d028607019,52006a9383bf149a4fb24226b173106f,shipped,2018-01-10 11:33:07,2018-01-11 02:32:30,2018-01-11 19:39:23,NaN,2018-02-07 00:00:00


Проверим, есть ли в наших данных, заказы с датой доставки  
(т.е. заказ доставлен, но отменен), удалим подобные набюдения.

In [21]:
undelivered_orders.order_delivered_customer_date.notna().sum()

6

In [22]:
drop_delivered = undelivered_orders[['order_id', 'order_delivered_customer_date']].dropna().order_id
undelivered_orders = undelivered_orders.query('order_id not in @drop_delivered')

In [23]:
undelivered_orders.order_delivered_customer_date.notna().sum()

0

Приведём колонку со временем покупки к формату месяц-год.

In [24]:
undelivered_orders['order_purchase_timestamp'] = undelivered_orders['order_purchase_timestamp'].dt.to_period('M')

In [25]:
undelivered_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04,2017-04-13 13:25:17,NaN,NaN,2017-05-09 00:00:00
44,ee64d42b8cf066f35eac1cf57de1aa85,caded193e8e47b8362864762a83db3c5,shipped,2018-06,2018-06-05 04:31:18,2018-06-05 14:32:00,NaN,2018-06-28 00:00:00
103,0760a852e4e9d89eb77bf631eaaf1c84,d2a79636084590b7465af8ab374a8cf5,invoiced,2018-08,2018-08-07 06:15:14,NaN,NaN,2018-08-21 00:00:00
128,15bed8e2fec7fdbadb186b57c46c92f2,f3f0e613e0bdb9c7cee75504f0f90679,processing,2017-09,2017-09-03 14:30:09,NaN,NaN,2017-10-03 00:00:00
154,6942b8da583c2f9957e990d028607019,52006a9383bf149a4fb24226b173106f,shipped,2018-01,2018-01-11 02:32:30,2018-01-11 19:39:23,NaN,2018-02-07 00:00:00


Создадим pivot-табоицу, где строки это месяцы, а столбцы причина  
недоставки (статус) заказа. Заполним NaN значения, для корректного  
подсчёта среднего значения.

In [26]:
undelivered_orders_by_month_and_status = undelivered_orders.groupby(['order_purchase_timestamp',
                                                                     'order_status'], as_index=False) \
                                                                     .agg({'order_id' : 'count'}) \
                                                                     .pivot(index='order_purchase_timestamp', 
                                                                            columns='order_status', values='order_id') \
                                                                     .fillna(0)

In [27]:
undelivered_orders_by_month_and_status

order_status,approved,canceled,created,invoiced,processing,shipped,unavailable
order_purchase_timestamp,,,,,,,
2016-09,0.0,2.0,0.0,0.0,0.0,1.0,0.0
2016-10,0.0,19.0,0.0,18.0,2.0,8.0,7.0
2017-01,0.0,3.0,0.0,12.0,9.0,16.0,10.0
2017-02,1.0,17.0,0.0,11.0,32.0,21.0,45.0
2017-03,0.0,33.0,0.0,3.0,23.0,45.0,32.0
2017-04,1.0,18.0,0.0,14.0,10.0,49.0,9.0
2017-05,0.0,29.0,0.0,16.0,23.0,55.0,31.0
2017-06,0.0,16.0,0.0,11.0,12.0,47.0,24.0
2017-07,0.0,28.0,0.0,7.0,11.0,56.0,52.0


Найдём среднее число недоставленных заказов в месяц, а также среднее число  
с детализацией по причинам.

In [28]:
undelivered_orders_by_month_and_status.mean().sum().round(2)

123.21

In [29]:
undelivered_orders_by_month_and_status.mean().round(2)

order_status
approved        0.08
canceled       25.79
created         0.21
invoiced       13.08
processing     12.54
shipped        46.12
unavailable    25.38
dtype: float64

### Задание 3

По каждому товару определить, в какой день недели товар чаще всего покупается.

Объединим таблицу с покупками (Задание 1) и с составом заказа.

In [30]:
purchases_items = purchases.merge(order_items, on='order_id', how='inner')

In [31]:
purchases_items.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,1,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-01-18 20:58:32,289.00,46.48
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00,1,bd07b66896d6f1494f5b86251848ced7,7c67e1448b00f6e969d365cea6b010ab,2018-06-05 16:19:10,139.94,17.79
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00,1,a5647c44af977b148e0a3a4751a09e2e,7c67e1448b00f6e969d365cea6b010ab,2018-03-27 16:31:16,149.94,23.36
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00,1,9391a573abe00141c56e38d84d7d5b3b,4a3ca9315b744ce9f8e9374361493884,2018-07-31 10:10:09,230.00,22.25


Создадим колонку с днём недели покупки.

In [32]:
purchases_items['purchase_day'] = purchases_items.order_purchase_timestamp \
                                                 .dt.day_name()

In [33]:
purchases_items.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,purchase_day
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,1,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88,Tuesday
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-01-18 20:58:32,289.00,46.48,Friday
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00,1,bd07b66896d6f1494f5b86251848ced7,7c67e1448b00f6e969d365cea6b010ab,2018-06-05 16:19:10,139.94,17.79,Saturday
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00,1,a5647c44af977b148e0a3a4751a09e2e,7c67e1448b00f6e969d365cea6b010ab,2018-03-27 16:31:16,149.94,23.36,Tuesday
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00,1,9391a573abe00141c56e38d84d7d5b3b,4a3ca9315b744ce9f8e9374361493884,2018-07-31 10:10:09,230.00,22.25,Sunday


Найдём самый успешный день для каждого товара по числу покупок.

In [34]:
product_top_day = purchases_items.groupby(['product_id', 'purchase_day'], as_index=False) \
                                 .agg({'order_id' : 'count'}) \
                                 .sort_values('order_id', ascending=False) \
                                 .drop_duplicates(subset='product_id') \
                                 .rename(columns={'order_id' : 'purchases'})

In [35]:
product_top_day

,product_id,purchase_day,purchases
15763,422879e10f46682990de24d770e7f83d,Wednesday,93
36172,99a4788cb24856965c36a24e339b6058,Monday,92
40673,aca2eb7d00ea1a7b8ebd4e68314663af,Thursday,89
19940,53b36df67ebb7c41585e8d54d6772e08,Tuesday,76
13419,389d119b48cf3043d311335e499d9c6b,Thursday,67
...,...,...,...
23808,643a4049ae3e6df63858575f28cd7b21,Friday,1
23810,643a66b1dc5dad3de6cb5a41549e72f1,Sunday,1
23812,643f86cfb81c8ef89293524c5cd78435,Thursday,1
23813,64436613065a92eedd50d29d4ec0cd43,Friday,1


### Задание 4

Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)?  
В метрике необходимо учесть не целое количество недель внутри месяца,  
например, в ноябре 2021 года 4,28 недели.

Приведём колонку со временем покупке к формату месяц-год.

In [36]:
purchases_items['order_purchase_timestamp'] = purchases_items['order_purchase_timestamp'].dt.to_period('M')

In [37]:
purchases_items.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,purchase_day
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,1,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88,Tuesday
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-01-18 20:58:32,289.00,46.48,Friday
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00,1,bd07b66896d6f1494f5b86251848ced7,7c67e1448b00f6e969d365cea6b010ab,2018-06-05 16:19:10,139.94,17.79,Saturday
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00,1,a5647c44af977b148e0a3a4751a09e2e,7c67e1448b00f6e969d365cea6b010ab,2018-03-27 16:31:16,149.94,23.36,Tuesday
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00,1,9391a573abe00141c56e38d84d7d5b3b,4a3ca9315b744ce9f8e9374361493884,2018-07-31 10:10:09,230.00,22.25,Sunday


Найдём число покупок для каждого клиента по месяцам, отсортируем  
по возрастанию месяца и id.

In [38]:
purchases_by_month = purchases_items.groupby(["customer_unique_id", "order_purchase_timestamp"], as_index=False) \
                                    .agg({"order_id" : "count"}) \
                                    .rename(columns={'order_id' : 'purchases'}) \
                                    .sort_values(['order_purchase_timestamp', 'customer_unique_id'])

In [39]:
purchases_by_month.head()

,customer_unique_id,order_purchase_timestamp,purchases
49538,830d5b7aaa3b6f1e9ad63703bec97d23,2016-09,3
69429,b7d76e111c89f7ebf14761390f0f7d17,2016-09,2
74,0032c76b20340da25249092a268ce66c,2016-10,1
741,01f156677184504063bd19739f924af1,2016-10,1
1166,0313291a6f4f16df04dcf819d88c38ef,2016-10,1


Создадим колонку с числом недель в месяц покупки.

In [40]:
purchases_by_month['weeks_in_purchase_month'] = purchases_by_month.order_purchase_timestamp \
                                                                  .dt.days_in_month \
                                                                  .div(7) \
                                                                  .round(2)

In [41]:
purchases_by_month.head()

,customer_unique_id,order_purchase_timestamp,purchases,weeks_in_purchase_month
49538,830d5b7aaa3b6f1e9ad63703bec97d23,2016-09,3,4.29
69429,b7d76e111c89f7ebf14761390f0f7d17,2016-09,2,4.29
74,0032c76b20340da25249092a268ce66c,2016-10,1,4.43
741,01f156677184504063bd19739f924af1,2016-10,1,4.43
1166,0313291a6f4f16df04dcf819d88c38ef,2016-10,1,4.43


Найдём среднее число покупок в неделю (по месяцам) для каждого пользователя.

In [42]:
purchases_by_month['avg_purchases_per_week_by_user'] = purchases_by_month.purchases \
                                                                         .div(purchases_by_month.weeks_in_purchase_month) \
                                                                         .round(2)

In [43]:
purchases_by_month

,customer_unique_id,order_purchase_timestamp,purchases,weeks_in_purchase_month,avg_purchases_per_week_by_user
49538,830d5b7aaa3b6f1e9ad63703bec97d23,2016-09,3,4.29,0.70
69429,b7d76e111c89f7ebf14761390f0f7d17,2016-09,2,4.29,0.47
74,0032c76b20340da25249092a268ce66c,2016-10,1,4.43,0.23
741,01f156677184504063bd19739f924af1,2016-10,1,4.43,0.23
1166,0313291a6f4f16df04dcf819d88c38ef,2016-10,1,4.43,0.23
...,...,...,...,...,...
96468,ffc4e1cd8aa220763480d0a18157c45d,2018-08,1,4.43,0.23
96499,ffe204b1bb8026741b839085b2fd10f2,2018-08,1,4.43,0.23
96520,ffec490ab531184a483efe2eedd68908,2018-08,1,4.43,0.23
96536,fff96bc586f78b1f070da28c4977e810,2018-08,1,4.43,0.23


### Задание 5

Провести когортный анализ пользователей. В период с января по декабрь выяви когорту  
с самым высоким retention на 3-ий месяц.

Создадим новый датафрейм на основе таблицы с покупками (Задание 1),  
приведём колонку со временем покупки к формату месяц-год.

In [44]:
# Используем copy(), чтобы не изменять исходный датафрейм
cohort_analysis = purchases.copy()

In [45]:
cohort_analysis['order_purchase_timestamp'] = cohort_analysis['order_purchase_timestamp'].dt.to_period('M')

In [46]:
cohort_analysis.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00


Рассмотрим за какой период данными мы располагаем.

In [47]:
cohort_analysis.order_purchase_timestamp \
               .value_counts() \
               .sort_index() \
               .index

PeriodIndex(['2016-09', '2016-10', '2016-12', '2017-01', '2017-02', '2017-03',
             '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09',
             '2017-10', '2017-11', '2017-12', '2018-01', '2018-02', '2018-03',
             '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09'],
            dtype='period[M]', name='order_purchase_timestamp')

В наших данных представлен лишь один календарный год. Создадим столбец  
с датой первой покупки (когортой) и отфильтруем наблюдения за 2017 год.

In [48]:
# Дата первой покупки(когорта)
cohort_analysis['cohort'] = cohort_analysis.groupby('customer_unique_id') \
                                           .order_purchase_timestamp \
                                           .transform('min')

In [49]:
cohorts = cohort_analysis[cohort_analysis.cohort.dt.year == 2017]

In [50]:
cohorts.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,cohort
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,2017-05
5,879864dab9bc3047522c92c82e1212b8,4c93744516667ad3b8f1fb645a3116a4,89254,jaragua do sul,SC,5741ea1f91b5fbab2bd2dc653a5b5099,delivered,2017-09,2017-09-14 18:25:11,2017-09-18 21:27:40,2017-09-28 17:32:43,2017-10-04 00:00:00,2017-09
7,5e274e7a0c3809e14aba7ad5aae0d407,57b2a98a409812fe9618067b6b8ebe4f,35182,timoteo,MG,1093c8304c7a003280dd34598194913d,delivered,2017-11,2017-11-16 19:55:41,2017-11-22 16:46:33,2017-11-27 12:44:36,2017-12-08 00:00:00,2017-11
10,9fb35e4ed6f0a14a4977cd9aea4042bb,2a7745e1ed516b289ed9b29c7d0539a5,39400,montes claros,MG,8428e578bb1cf839ae26a6b7615502b9,delivered,2017-11,2017-11-27 18:01:20,2017-11-28 23:04:41,2017-12-08 22:16:17,2017-12-20 00:00:00,2017-11
12,b2d1536598b73a9abd18e0d75d92f0a3,918dc87cd72cd9f6ed4bd442ed785235,18682,lencois paulista,SP,83d8f70418eefd96f37b483dff430305,delivered,2017-09,2017-09-09 10:10:17,2017-09-12 12:42:43,2017-09-20 20:23:34,2017-10-02 00:00:00,2017-09


Вычислим число пользователей в каждой когорте в каждом наблюдаемом месяце.

In [51]:
cohorts = cohorts.groupby(['cohort', 'order_purchase_timestamp'],
                          as_index=False) \
                 .agg({'customer_unique_id': pd.Series.nunique})

In [52]:
cohorts.head()

,cohort,order_purchase_timestamp,customer_unique_id
0,2017-01,2017-01,738
1,2017-01,2017-02,3
2,2017-01,2017-03,2
3,2017-01,2017-04,1
4,2017-01,2017-05,3


Вычислим номер наблюдаемого месяца (период существования когорты).

In [53]:
cohorts['period'] = cohorts.order_purchase_timestamp.astype('int64') - cohorts.cohort.astype('int64')

In [54]:
cohorts.head()

,cohort,order_purchase_timestamp,customer_unique_id,period
0,2017-01,2017-01,738,0
1,2017-01,2017-02,3,1
2,2017-01,2017-03,2,2
3,2017-01,2017-04,1,3
4,2017-01,2017-05,3,4


Создадим pivot-таблицу, где строки это когорты (месяц первой покупки),  
а столбцы период существования когорты (в месяцах).

In [55]:
cohorts_pivot = cohorts.pivot(index='cohort', columns = 'period', values = 'customer_unique_id')

In [56]:
cohorts_pivot

period,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19
cohort,,,,,,,,,,,,,,,,,,,
2017-01,738.0,3.0,2.0,1.0,3.0,1.0,3.0,1.0,1.0,NaN,3.0,1.0,5.0,3.0,1.0,1.0,2.0,3.0,1.0
2017-02,1667.0,4.0,5.0,2.0,7.0,2.0,4.0,3.0,1.0,3.0,2.0,5.0,2.0,3.0,2.0,1.0,1.0,4.0,NaN
2017-03,2568.0,13.0,9.0,10.0,9.0,4.0,4.0,8.0,9.0,2.0,9.0,3.0,6.0,3.0,4.0,6.0,2.0,4.0,NaN
2017-04,2311.0,14.0,5.0,4.0,7.0,6.0,8.0,7.0,7.0,4.0,6.0,2.0,1.0,1.0,2.0,2.0,3.0,NaN,NaN
2017-05,3525.0,17.0,17.0,14.0,11.0,12.0,15.0,5.0,9.0,9.0,9.0,12.0,9.0,1.0,7.0,8.0,NaN,NaN,NaN
2017-06,3094.0,14.0,11.0,12.0,8.0,11.0,11.0,7.0,4.0,7.0,10.0,10.0,5.0,4.0,6.0,NaN,NaN,NaN,NaN
2017-07,3815.0,20.0,13.0,9.0,11.0,8.0,12.0,4.0,7.0,10.0,8.0,11.0,5.0,10.0,NaN,NaN,NaN,NaN,NaN
2017-08,4112.0,28.0,14.0,11.0,15.0,22.0,12.0,11.0,6.0,6.0,10.0,8.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
2017-09,4060.0,28.0,22.0,12.0,18.0,9.0,9.0,10.0,12.0,7.0,11.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Найдём когорту с максимальным retention на 3 месяц и её retention.

In [57]:
(cohorts_pivot[3] / cohorts_pivot[0] * 100).idxmax()

Period('2017-05', 'M')

In [58]:
(cohorts_pivot[3] / cohorts_pivot[0] * 100).max().round(3)

0.397

### Задание 6

Построить RFM-сегментацию пользователей.

Объединяем таблицу с покупками (Задание 1) и с составом заказа.  
Затем отбираем необходимые столбцы и приводим колонку со временем  
покупки к формату день-месяц-год.

In [59]:
rfm_analysis = purchases.merge(order_items, on='order_id', how='inner')

In [60]:
rfm_analysis = rfm_analysis[['order_id', 'customer_unique_id',
                             'order_purchase_timestamp', 'price']]

In [61]:
rfm_analysis['order_purchase_timestamp'] = rfm_analysis['order_purchase_timestamp'].dt.to_period('D')

In [62]:
rfm_analysis.head()

,order_id,customer_unique_id,order_purchase_timestamp,price
0,00e7ee1b050b8499577073aeb2a297a1,861eff4711a542e4b93843c6dd7febb0,2017-05-16,124.99
1,29150127e6685892b6eab3eec79f59c7,290c77bc529b7ac935b93aa66c333dc3,2018-01-12,289.00
2,b2059ed67ce144a36e2aa97d2c9e9ad2,060e732b5b29e8181a18229c7b0b2b5e,2018-05-19,139.94
3,951670f92359f4fe4a63112aa7306eba,259dac757896d24d7702b9acbbff3f3c,2018-03-13,149.94
4,6b7d50bd145f6fc7f33cebabd7e49d0f,345ecd01c38d18a9036ed96c73b8d066,2018-07-29,230.00


Создаем переменную с максимальным числом дней (последним заказом)  
для расчёта recency (дней с последней сделки), затем подсчитываем frequency  
(количество покупок) и monetary (сумму покупок) для каждого пользователя.

In [63]:
last_day = rfm_analysis.order_purchase_timestamp \
                       .sort_values(ascending=False) \
                       .astype('int64').max()

In [64]:
recency = last_day - rfm_analysis.groupby('customer_unique_id') \
                                 .order_purchase_timestamp \
                                 .max() \
                                 .astype('int64') \
                                 .rename('days_from_last_purchase')

In [65]:
frequency  = rfm_analysis.order_id \
                         .groupby(rfm_analysis.customer_unique_id) \
                         .nunique() \
                         .rename('purchases')

In [66]:
monetary  = rfm_analysis.groupby('customer_unique_id') \
                        .price \
                        .sum() \
                        .rename('total_check')

Делим показатели recency и monetary на 5 групп при помощи квантилей.  
С frequency (числом покупок) так не получится: об этом далее.

In [67]:
recency_seg = pd.qcut(recency, q=5,
                      labels=[5, 4, 3, 2, 1]) \
                .rename('recency_seg')

In [68]:
monetary_seg = pd.qcut(monetary, q=5,
                       labels=[1, 2, 3, 4, 5]) \
                 .rename('monetary_seg')

frequency мы не можем разделить по квантилям из-за большго хвоста значений  
с 1, 2 и 3 покупками. Поэтому разделим их вручную.

In [69]:
frequency.value_counts()

purchases
1     91803
2      2636
3       187
4        29
5         9
6         5
7         3
9         1
16        1
Name: count, dtype: int64

In [70]:
frequency_seg = pd.cut(frequency, bins=[0, 1, 2, 3, 4, 1000],
                       labels=[1, 2, 3, 4, 5]) \
                  .rename('frequency_seg')

Объединим следующие pd.Series по customer_unique_id(индекс): 

1. Число дней с последней покупки;
2. Число покупок;
3. Сумма покупок;
4. Сегмент recency;
5. Сегмент frequency;
6. Сегмент monetary.

Также создадим колонку с RFM-сегентом.

In [71]:
rfm = pd.concat([recency, frequency, monetary, recency_seg,
                 frequency_seg, monetary_seg], axis=1) \
        .reset_index()

In [72]:
rfm['RFM'] = rfm.recency_seg.astype(str) + rfm.frequency_seg.astype(str) + rfm.monetary_seg.astype(str)

Итоговая RFM-таблица:

* Recency — чем больше цифра, тем недавнее последняя покупка  
(days_from_last_purchase);  
* Frequency — чем больше цифра, тем больше покупок (purchases);  
* Monetary — чем больше цифра, тем больше общая сумма покупок  
(total_check).

Следовательно, чем RFM ближе к 555, тем более активен пользователь и наоборот.

In [73]:
rfm

,customer_unique_id,days_from_last_purchase,purchases,total_check,recency_seg,frequency_seg,monetary_seg,RFM
0,0000366f3b9a7992bf8c76cfdf3221e2,116,1,129.90,4,1,4,414
1,0000b849f77a49e4a4ce2b2a4ca5be3f,119,1,18.90,4,1,1,411
2,0000f46a3911fa3c0805444483337064,542,1,69.00,1,1,2,112
3,0000f6ccb0745a6a4b88665a16c9f078,326,1,25.99,2,1,1,211
4,0004aac84e0df4da2b147fca70cf8255,293,1,180.00,2,1,5,215
...,...,...,...,...,...,...,...,...
94669,fffcf5a5ff07b0908bd4e2dbc735a684,452,1,1570.00,1,1,5,115
94670,fffea47cd6d3cc0a88bd621562a9d061,267,1,64.89,3,1,2,312
94671,ffff371b4d645b6ecea244b27531430a,573,1,89.90,1,1,3,113
94672,ffff5962728ec6157033ef9805bacc48,124,1,115.00,4,1,4,414
